# Homework 1. MNIST 데이터셋을 활용해 다중 분류 수행하기
### 과목코드: [입력하시오] (2150034001/2150085101)

#### 문제. 아래의 `코드`를 완성하고, 그에 대한 `분석`을 작성하시오.

1. (코드) MNIST 데이터셋 받기
   - (분석) 데이터셋에 대해 분석하기

2. (코드) 데이터셋을 train / validation / test set 으로 분할하기
   - (분석) 데이터셋을 분할한 방식에 대해 설명하고, 해당 분할 방식을 수행한 이유에 대해 설명하기

3. (코드) 특성 스케일링을 이용한 데이터셋 전처리하기
   - (분석) 스케일링을 수행한 방식에 대해 설명하고, 해당 스케일링을 수행한 이유에 대해 설명하기

4. (코드) 모델 선택하기
   - (분석) 머신러닝 모델의 후보군들에 대해 설명하고, 후보군들 중 특정 모델이 대표 모델로 선택된 이유에 대해 설명하기

1. 로지스틱 회귀 (Logistic Regression)
원래 binary 분류지만 softmax Regression 사용 시 다중 클래스 분류에도 사용 가능
근데 mnist는 데이터가 비선형적임 (4와 9의 유사성 등) 

2. kNN, 최근접이웃 (k-Nearest Neighbors) 
새로운 데이터 = 가장 가까운 k개의 이웃의 class를 참조해 분류
scikit learn 으로 구현 사례 존재 (https://teddylee777.github.io/scikit-learn/scikit-learn-knn/)

3. CNN 
 이미지 처리에 자주 사용되며 복잡한 패턴 인식에 효율적이라고 함, 또 MNIST 데이터셋 분류 성공률이 제일 높다고 함 (https://paperswithcode.com/sota/image-classification-on-mnist)
 구현하고 설명하는게 쉽지 않아보임

5. (코드) 성능 평가하기
   - (분석) 최종적으로 학습된 모델을 이용해 test set 에 대한 성능을 정리하고, 이를 분석하기

6. (코드) 에러 분석하기
   - (분석) 최종적으로 학습된 모델의 에러를 분석하기
   - (분석) `5. 성능 평가하기` 의 분석과 연관지어 에러를 분석하고 해결책을 제안하기

참고자료